In [221]:
import networkx as nx
import matplotlib.pyplot as plt 
from pymystem3 import Mystem as mystem
import re
from nltk.tokenize import RegexpTokenizer as rt
from stop_words import get_stop_words
stop_words = get_stop_words('ru')
from nltk import FreqDist
import os
import itertools
import json

## Создаем список текстов

In [222]:
list_of_texts = []
for some_file in os.listdir ('./Corpus'): 
    if some_file.endswith ('.txt'):
        with open (os.path.join('./Corpus', some_file),'r') as open_file:
            list_of_texts.append(open_file.read())

In [198]:
list_of_texts

['Рассказ этот — истинное происшествие. Случилось в Астрахани. Рассказал мне об этом актёр-любитель.\nВот что он рассказал:\n«Вот вы меня, граждане, спрашиваете, был ли я актёром? Ну, был. В театре играл. Прикасался к этому искусству. А только ерунда. Ничего в этом нет выдающего.\nКонечно, если подумать глубже, то в этом искусстве много хорошего.\nСкажем, выйдешь на сцену, а публика смотрит. А средь публики — знакомые, родственники со стороны жены, граждане с дому. Глядишь — подмигивают с партеру — дескать, не робей, Вася, дуй до горы. А ты, значит, им знаки делаешь — дескать, оставьте беспокоиться, граждане. Знаем. Сами с усами.\nНо если подумать глубже, то ничего в этой профессии нету хорошего. Крови больше испортишь.\nВот раз ставили мы пьесу «Кто виноват». Из прежней жизни. Очень это сильная пьеса. Там, значит, в одном акте грабители купца грабят на глазах у публики. Очень натурально выходит. Купец, значит, кричит, ногами отбивается. А его грабят. Жуткая пьеса.\nТак вот, поставили 

## Создаем функцию для лемматизации одного текста

In [223]:
def lemmatization_of_one_text(text):
    mystem = Mystem()
    lemmas = mystem.lemmatize(text)
    for word_idx in range(len(lemmas)):
        lemmas[word_idx] = lemmas[word_idx].strip('\n').strip('\xa0 ')
    lemmas = ' '.join(lemmas)
    clean_lemmas = re.sub('\d+?', '', lemmas)
    return clean_lemmas

## Создадим функцию, которая пролемматизирует нам все тексты

In [224]:
def lemmas_of_all_texts(list_of_texts):
    lemmas_of_all_texts = []
    for i in range(len(list_of_texts)):
        lemmas_of_all_texts.append(lemmatization(list_of_texts[i]))
    return lemmas_of_all_texts   

In [225]:
list_of_lemmas = lemmas_of_all_texts(list_of_texts)

## То же самое создадим для токенизации

In [226]:
def tokenization(text):
    tokenizer = rt('\w+')
    tokenized_lemmas = tokenizer.tokenize(text)
    return tokenized_lemmas

In [227]:
def tokenization_of_all_texts(list_of_lemmas):
    tokens_of_all_texts = []
    for i in range(len(list_of_texts)):
        tokens_of_all_texts.append(tokenization(list_of_lemmas[i]))
    return tokens_of_all_texts   

In [228]:
list_of_tokenized_texts = tokenization_of_all_texts(list_of_lemmas)

## Теперь почистим от стоп-слов (в какой-то степени это опционально, особенно, если мы работаем с биграмами)

In [229]:
def drop_stop_words_from_text_list(list_of_tokenized_texts):
    for i in range(len(list_of_tokenized_texts)):
        for stop_word in stop_words:
            list_of_tokenized_texts[i] = list(filter(lambda word: word != stop_word, list_of_tokenized_texts[i]))
    return list_of_tokenized_texts

In [230]:
no_stop_words = drop_stop_words_from_text_list(list_of_tokenized_texts)

In [231]:
zoshenko = no_stop_words[0]

In [232]:
QGIS = no_stop_words[1]

In [233]:
N_1 = no_stop_words[2]

In [234]:
Gus = no_stop_words[3]

## Попробуем создать пары слов для каждого нашего текста

In [235]:
def couple(list_of_tokens):
    couples = []
    for i, word in enumerate(list_of_tokens[:-1]):
        couple = word + ' '+ list_of_tokens[i+1]
        couples.append(couple)
    return couples

In [236]:
zoshenko_couples = couple(list_of_tokenized_texts[0])

In [237]:
QGIS_couples  = couple(list_of_tokenized_texts[1])

In [238]:
N_1_couples  = couple(list_of_tokenized_texts[2])

In [239]:
Gus_couples  = couple(list_of_tokenized_texts[3])

## Теперь создадим словари частотности. Первый подходит просто для подсчета слов, в то время как второй нужен для биграмов, причем в нем дуплеты типа "он пошел" и "пошел он" слиты воедино

In [240]:
def counting_words(list_of_words):
    dict_with_quantity = {}
    for word in list_of_words:
        if word in dict_with_quantity:
            dict_with_quantity[word] +=1
        else:
            dict_with_quantity[word]=1
    return dict_with_quantity

In [241]:
def sum_similar_pair_values(pair_dic):
    filter_pair_dic = {}
    for key1 in pair_dic.keys():
        for key2 in pair_dic.keys():
            if key1[0] == key2[1] and key1[1] == key2[0] and key1[0] < key1[1]:
                filter_pair_dic[key1] = pair_dic[key1] + pair_dic[key2]
    return filter_pair_dic

In [242]:
def sort(final_dictionary):
    itog = dict(sorted(final_dictionary.items(), key=lambda x: x[1], reverse=True))
    return itog

In [243]:
def dict_slice(dictionary, index):
    sliced_dictionary = dict(itertools.islice(sort(counting_words(dictionary)).items(), index))
    return sliced_dictionary

In [244]:
def dict_of_couples_slice(dictionary, index):
    sliced_dictionary = dict(itertools.islice(sort(sum_similar_pair_values(counting_words(dictionary))).items(), index))
    return sliced_dictionary

## Итак, время посмотреть на частотности!

### Частотности для одиноких слов

In [245]:
dict_slice(zoshenko, 10)

{'говорить': 24,
 'выходить': 9,
 'пьеса': 9,
 'гражданин': 7,
 'публика': 6,
 'купец': 6,
 'кричать': 6,
 'любитель': 5,
 'играть': 4,
 'дескать': 4}

In [246]:
dict_slice(N_1, 10)

{'экзопланета': 11,
 'проксим': 8,
 'звезда': 6,
 'обнаруживать': 6,
 'астроном': 5,
 'наблюдение': 5,
 'объект': 4,
 'кандидат': 4,
 'помощь': 4,
 'телескоп': 3}

In [247]:
dict_slice(QGIS, 10)

{'карта': 15,
 'данные': 10,
 'гис': 9,
 'проекция': 7,
 'система': 6,
 'работа': 5,
 'например': 5,
 'изображение': 5,
 'QGIS': 4,
 'использоваться': 4}

In [248]:
dict_slice(Gus, 10)

{'гусь': 64,
 'птица': 10,
 'конец': 7,
 'роспись': 7,
 'гусыня': 7,
 'город': 7,
 'римлянин': 6,
 'лебедь': 6,
 'сократ': 6,
 'матушка': 6}

### Частотности для пар слов

In [249]:
dict_of_couples_slice(zoshenko_couples, 10)

{'виноватый прежний': 4,
 'видеть роль': 4,
 'публика дура': 4,
 'видеть любитель': 4,
 'видеть крик': 4,
 'видеть пьеса': 4,
 'ерунда ничто': 3,
 'истинный происшествие': 2,
 'астрахань рассказывать': 2,
 'актер любитель': 2}

In [250]:
dict_of_couples_slice(N_1_couples, 10)

{'астроном обнаруживать': 3,
 'астроном подводить': 2,
 'комплексный наблюдение': 2,
 'опубликовывать сайт': 2,
 'существование звезда': 2,
 'суперземля минимальный': 2,
 'орбитальный период': 2,
 'большой полуось': 2,
 'орбита астрономический': 2,
 'астрономический единица': 2}

In [251]:
dict_of_couples_slice(QGIS_couples, 10)

{'анализ статистический': 3,
 'горный высота': 2,
 'большой данные': 2,
 'использоваться случай': 2,
 'оставаться связанный': 2,
 'исследовать мнение': 2,
 'официальный статистический': 2,
 'использоваться разный': 2,
 'отрасль литература': 2,
 'источник черпать': 2}

In [252]:
dict_of_couples_slice(Gus_couples, 10)

{'дикий гусь': 4,
 'зимний солнцестояние': 3,
 'конец лето': 3,
 'домашний гусь': 3,
 'история рим': 3,
 'конец конец': 3,
 'город гусь': 3,
 'осенний равноденствие': 2,
 'весенний возвращаться': 2,
 'зима дельта': 2}

## Финальные функции для текстов:

In [301]:
def counter(list_of_texts):
    list_of_lemmatized_texts = lemmas_of_all_texts(list_of_texts)
    list_of_tokenized_texts = tokenization_of_all_texts(list_of_lemmatized_texts)
    no_stop_words = drop_stop_words_from_text_list(list_of_tokenized_texts)
    counted = []
    for i in range(len(no_stop_words)):
        counted.append(counting_words(no_stop_words[i]))
    return counted

In [341]:
def sorter(dictionaries):
    for i in range(len(dictionaries)):
        dictionaries[i] = sort(dictionaries[i])
    return dictionaries

## TEST RUSSIA TODAY #1

In [254]:
russia_today = []
with open("russia_today_1.jsonlines", "r", encoding="UTF-8") as read_file:
    for line in read_file:
        russia_today.append(json.loads(line))

In [365]:
list_of_content = []
for item in russia_today[:4]:
    list_of_content.append(item['content'])

In [373]:
dictionaries = counter(list_of_content)

In [367]:
dictions = sorter(dictionaries)

In [369]:
for i in range(len(dictions)):
    print('Ключевые слова:', dict(itertools.islice(dictions[i].items(), 20)))

Ключевые слова: {'команда': 23, 'болельщик': 12, 'клуб': 11, 'приходиться': 10, 'сезон': 10, 'лига': 9, 'игрок': 8, 'россия': 7, 'новый': 7, 'матч': 7, 'спартак': 6, 'чемпионат': 6, 'зима': 6, 'европа': 6, 'решать': 6, 'свой': 5, 'зенит': 5, 'сделать': 5, 'вернуться': 5, 'цска': 5}
Ключевые слова: {'нэшвилл': 12, 'питтсбург': 11, 'команда': 10, 'матч': 9, 'серия': 7, 'кубок': 6, 'стэнли': 6, 'становиться': 5, 'игра': 5, 'выигрывать': 5, 'защитник': 5, 'гость': 5, 'NHL': 5, 'июнь': 4, 'история': 4, 'свой': 4, 'встреча': 4, 'большинство': 4, 'бросок': 4, 'гордость': 3}
Ключевые слова: {'сша': 13, 'российский': 12, 'сми': 11, 'россия': 8, 'иностранный': 7, 'американский': 7, 'мера': 7, 'агент': 6, 'отношение': 6, 'RT': 6, 'закон': 5, 'москва': 5, 'посольство': 4, 'вашингтон': 4, 'телеканал': 4, 'работа': 4, 'ответ': 4, 'принимать': 4, 'журналист': 4, 'миротворец': 4}
Ключевые слова: {'биткоин': 20, 'криптовалюта': 11, 'тыс': 9, 'цена': 8, 'декабрь': 6, 'курс': 5, 'валюта': 5, 'крупный': 4